# Advanced RAG Patterns with Egnyte-LangChain

This notebook demonstrates advanced Retrieval-Augmented Generation (RAG) patterns using Egnyte as the knowledge base and LangChain for AI processing.

## Advanced Patterns Covered

1. **Multi-Query RAG**: Generate multiple queries for comprehensive retrieval
2. **Hierarchical RAG**: Organize documents by importance and relevance
3. **Contextual Compression**: Compress retrieved content for better AI processing
4. **Self-Querying**: Let AI determine search parameters
5. **Ensemble Retrieval**: Combine multiple retrieval strategies

In [ ]:
# Setup and imports
import os
from dotenv import load_dotenv
from typing import List, Dict, Any

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import (
    MultiQueryRetriever,
    ContextualCompressionRetriever,
    EnsembleRetriever
)
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document

# Egnyte imports
from langchain_egnyte import EgnyteRetriever, EgnyteSearchOptions

# Load environment
load_dotenv()

EGNYTE_DOMAIN = os.getenv("EGNYTE_DOMAIN")
EGNYTE_USER_TOKEN = os.getenv("EGNYTE_USER_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("Advanced RAG Setup Complete")

## 1. Multi-Query RAG

Generate multiple related queries to improve retrieval coverage:

In [ ]:
# Initialize base components
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
base_retriever = EgnyteRetriever(
    domain=EGNYTE_DOMAIN,
    user_token=EGNYTE_USER_TOKEN
)

# Create multi-query retriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever,
    llm=llm,
    verbose=True
)

print("Multi-Query Retriever created")

In [ ]:
# Test multi-query retrieval
user_question = "What are the financial projections for next quarter?"

print(f"Original Question: {user_question}")
print("\nMulti-Query Retrieval in progress...")

# This will generate multiple related queries automatically
multi_docs = multi_query_retriever.invoke(user_question)

print(f"\nRetrieved {len(multi_docs)} documents from multiple queries")
for i, doc in enumerate(multi_docs[:3], 1):
    print(f"{i}. {doc.metadata.get('name', 'Unknown')} - {doc.metadata.get('path', 'Unknown')}")

## 2. Contextual Compression RAG

Compress retrieved documents to focus on relevant content:

In [ ]:
# Create contextual compression retriever
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

print("Contextual Compression Retriever created")

In [ ]:
# Compare regular vs compressed retrieval
query = "project budget allocation"

# Regular retrieval
regular_docs = base_retriever.invoke(query)
print(f"Regular Retrieval: {len(regular_docs)} documents")
if regular_docs:
    print(f"   First doc length: {len(regular_docs[0].page_content)} characters")

# Compressed retrieval
compressed_docs = compression_retriever.invoke(query)
print(f"\nCompressed Retrieval: {len(compressed_docs)} documents")
if compressed_docs:
    print(f"   First doc length: {len(compressed_docs[0].page_content)} characters")
    print(f"   Compression ratio: {len(compressed_docs[0].page_content) / len(regular_docs[0].page_content):.2%}")

## 3. Hierarchical Document Organization

Organize documents by relevance and importance:

In [ ]:
class HierarchicalRetriever:
    """Custom retriever that organizes documents hierarchically."""
    
    def __init__(self, base_retriever: EgnyteRetriever, llm: ChatOpenAI):
        self.base_retriever = base_retriever
        self.llm = llm
        
    def _score_relevance(self, doc: Document, query: str) -> float:
        """Score document relevance using LLM."""
        prompt = f"""
        Rate the relevance of this document to the query on a scale of 0-1.
        
        Query: {query}
        Document: {doc.page_content[:500]}...
        
        Relevance score (0-1):
        """
        
        try:
            response = self.llm.invoke(prompt)
            score = float(response.content.strip())
            return max(0, min(1, score))  # Clamp to 0-1
        except:
            return 0.5  # Default score
    
    def retrieve_hierarchical(self, query: str, max_docs: int = 10) -> Dict[str, List[Document]]:
        """Retrieve and organize documents hierarchically."""
        
        # Get base documents
        docs = self.base_retriever.invoke(query)
        
        # Score each document
        scored_docs = []
        for doc in docs[:max_docs]:
            score = self._score_relevance(doc, query)
            scored_docs.append((doc, score))
        
        # Sort by score
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        # Organize into tiers
        hierarchy = {
            "high_relevance": [doc for doc, score in scored_docs if score >= 0.8],
            "medium_relevance": [doc for doc, score in scored_docs if 0.5 <= score < 0.8],
            "low_relevance": [doc for doc, score in scored_docs if score < 0.5]
        }
        
        return hierarchy

# Create hierarchical retriever
hierarchical_retriever = HierarchicalRetriever(base_retriever, llm)
print("Hierarchical Retriever created")

In [ ]:
# Test hierarchical retrieval
query = "quarterly sales performance"
hierarchy = hierarchical_retriever.retrieve_hierarchical(query)

print(f"Hierarchical Results for: '{query}'")
print(f"\nHigh Relevance: {len(hierarchy['high_relevance'])} documents")
for doc in hierarchy['high_relevance']:
    print(f"   - {doc.metadata.get('name', 'Unknown')}")

print(f"\nMedium Relevance: {len(hierarchy['medium_relevance'])} documents")
for doc in hierarchy['medium_relevance'][:3]:  # Show first 3
    print(f"   - {doc.metadata.get('name', 'Unknown')}")

print(f"\nLow Relevance: {len(hierarchy['low_relevance'])} documents")

## 4. Self-Querying Retriever

Let the AI determine optimal search parameters:

In [ ]:
class SelfQueryingEgnyteRetriever:
    """Self-querying retriever that optimizes search parameters."""
    
    def __init__(self, domain: str, user_token: str, llm: ChatOpenAI):
        self.domain = domain
        self.user_token = user_token
        self.llm = llm
    
    def _generate_search_strategy(self, user_query: str) -> Dict[str, Any]:
        """Generate optimal search strategy based on user query."""
        
        strategy_prompt = f"""
        Analyze this user query and suggest optimal search parameters for an enterprise document search.
        
        User Query: "{user_query}"
        
        Suggest:
        1. Refined search terms (comma-separated)
        2. Likely folder paths (if any specific departments/projects mentioned)
        3. Document limit (5-20 based on query complexity)
        4. Time relevance (recent, any, or specific period)
        
        Format your response as:
        TERMS: [refined terms]
        FOLDERS: [folder paths or "any"]
        LIMIT: [number]
        TIME: [recent/any/specific]
        """
        
        response = self.llm.invoke(strategy_prompt)
        return self._parse_strategy_response(response.content)
    
    def _parse_strategy_response(self, response: str) -> Dict[str, Any]:
        """Parse LLM strategy response."""
        strategy = {
            "terms": None,
            "folders": None,
            "limit": 10,
            "time_filter": None
        }
        
        lines = response.strip().split('\n')
        for line in lines:
            if line.startswith('TERMS:'):
                strategy['terms'] = line.replace('TERMS:', '').strip()
            elif line.startswith('FOLDERS:'):
                folders = line.replace('FOLDERS:', '').strip()
                strategy['folders'] = None if folders.lower() == 'any' else folders
            elif line.startswith('LIMIT:'):
                try:
                    strategy['limit'] = int(line.replace('LIMIT:', '').strip())
                except:
                    strategy['limit'] = 10
        
        return strategy
    
    def retrieve(self, user_query: str) -> List[Document]:
        """Perform self-querying retrieval."""
        
        # Generate search strategy
        strategy = self._generate_search_strategy(user_query)
        print(f"AI Strategy: {strategy}")
        
        # Create search options based on strategy
        search_options = EgnyteSearchOptions(
            limit=strategy['limit']
        )
        
        if strategy['folders']:
            search_options.folder_path = strategy['folders']
        
        # Create retriever with optimized options
        retriever = EgnyteRetriever(
            domain=self.domain,
            user_token=self.user_token,
            search_options=search_options
        )
        
        # Use refined terms or original query
        search_query = strategy['terms'] if strategy['terms'] else user_query
        
        return retriever.invoke(search_query)

# Create self-querying retriever
self_query_retriever = SelfQueryingEgnyteRetriever(
    domain=EGNYTE_DOMAIN,
    user_token=EGNYTE_USER_TOKEN,
    llm=llm
)

print("Self-Querying Retriever created")

In [ ]:
# Test self-querying
complex_query = "I need to find information about our marketing campaign performance from the last quarter, specifically focusing on digital channels and ROI metrics"

print(f"Complex Query: {complex_query}")
print("\nAI is analyzing and optimizing search strategy...")

self_query_docs = self_query_retriever.retrieve(complex_query)

print(f"\nSelf-Query Results: {len(self_query_docs)} documents")
for i, doc in enumerate(self_query_docs[:3], 1):
    print(f"{i}. {doc.metadata.get('name', 'Unknown')} - {doc.metadata.get('path', 'Unknown')}")

## 5. Ensemble Retrieval

Combine multiple retrieval strategies for optimal results:

In [ ]:
# Create multiple specialized retrievers
recent_retriever = EgnyteRetriever(
    domain=EGNYTE_DOMAIN,
    user_token=EGNYTE_USER_TOKEN,
    search_options=EgnyteSearchOptions(
        limit=5,
        created_after=datetime.now() - timedelta(days=30)
    )
)

comprehensive_retriever = EgnyteRetriever(
    domain=EGNYTE_DOMAIN,
    user_token=EGNYTE_USER_TOKEN,
    search_options=EgnyteSearchOptions(limit=15)
)

# Create ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[recent_retriever, comprehensive_retriever, compression_retriever],
    weights=[0.3, 0.4, 0.3]  # Weight recent docs, comprehensive search, and compressed content
)

print("Ensemble Retriever created with 3 strategies")

In [ ]:
# Test ensemble retrieval
ensemble_query = "team productivity metrics"

print(f"Ensemble Query: {ensemble_query}")
print("\nRunning multiple retrieval strategies...")

ensemble_docs = ensemble_retriever.invoke(ensemble_query)

print(f"\nEnsemble Results: {len(ensemble_docs)} documents")
print("\nTop Results:")
for i, doc in enumerate(ensemble_docs[:5], 1):
    print(f"{i}. {doc.metadata.get('name', 'Unknown')}")
    print(f"   Path: {doc.metadata.get('path', 'Unknown')}")
    print(f"   Size: {doc.metadata.get('size', 'Unknown')} bytes")

## 6. Advanced RAG Chain with All Patterns

Combine all patterns into a sophisticated RAG system:

In [ ]:
class AdvancedRAGSystem:
    """Advanced RAG system combining multiple retrieval patterns."""
    
    def __init__(self, domain: str, user_token: str, llm: ChatOpenAI):
        self.domain = domain
        self.user_token = user_token
        self.llm = llm
        
        # Initialize all retrievers
        self.base_retriever = EgnyteRetriever(domain, user_token)
        self.multi_query = MultiQueryRetriever.from_llm(self.base_retriever, llm)
        self.hierarchical = HierarchicalRetriever(self.base_retriever, llm)
        self.self_query = SelfQueryingEgnyteRetriever(domain, user_token, llm)
        
        # Create advanced prompt
        self.prompt = PromptTemplate(
            template="""
            You are an expert analyst with access to enterprise documents from Egnyte.
            
            CONTEXT FROM MULTIPLE RETRIEVAL STRATEGIES:
            {context}
            
            QUESTION: {question}
            
            Provide a comprehensive analysis that:
            1. Synthesizes information from all retrieved documents
            2. Identifies key insights and patterns
            3. Highlights any conflicting information
            4. Provides actionable recommendations
            5. Cites specific documents for each claim
            
            ANALYSIS:
            """,
            input_variables=["context", "question"]
        )
    
    def analyze(self, question: str) -> Dict[str, Any]:
        """Perform advanced RAG analysis."""
        
        print(f"Analyzing: {question}")
        
        # 1. Multi-query retrieval
        print("\nMulti-Query Retrieval...")
        multi_docs = self.multi_query.invoke(question)
        
        # 2. Hierarchical organization
        print("Hierarchical Organization...")
        hierarchy = self.hierarchical.retrieve_hierarchical(question)
        
        # 3. Self-querying optimization
        print("Self-Querying Optimization...")
        self_query_docs = self.self_query.retrieve(question)
        
        # Combine all documents
        all_docs = multi_docs + hierarchy['high_relevance'] + self_query_docs
        
        # Remove duplicates based on path
        unique_docs = []
        seen_paths = set()
        for doc in all_docs:
            path = doc.metadata.get('path', '')
            if path not in seen_paths:
                unique_docs.append(doc)
                seen_paths.add(path)
        
        print(f"\nCombined {len(unique_docs)} unique documents")
        
        # Create context
        context = "\n\n".join([
            f"Document: {doc.metadata.get('name', 'Unknown')}\n{doc.page_content}"
            for doc in unique_docs[:10]  # Limit to top 10 for context window
        ])
        
        # Generate analysis
        print("Generating AI Analysis...")
        analysis_prompt = self.prompt.format(context=context, question=question)
        analysis = self.llm.invoke(analysis_prompt)
        
        return {
            "question": question,
            "analysis": analysis.content,
            "source_documents": unique_docs,
            "document_count": len(unique_docs),
            "hierarchy": {
                "high": len(hierarchy['high_relevance']),
                "medium": len(hierarchy['medium_relevance']),
                "low": len(hierarchy['low_relevance'])
            }
        }

# Create advanced RAG system
advanced_rag = AdvancedRAGSystem(
    domain=EGNYTE_DOMAIN,
    user_token=EGNYTE_USER_TOKEN,
    llm=llm
)

print("Advanced RAG System initialized")

In [ ]:
# Test the advanced RAG system
complex_business_question = "What are the key challenges and opportunities in our current market position, and what strategic recommendations can be made based on recent performance data?"

result = advanced_rag.analyze(complex_business_question)

print("\n" + "="*80)
print("ADVANCED RAG ANALYSIS RESULTS")
print("="*80)

print(f"\nAnalysis Summary:")
print(f"   Documents Analyzed: {result['document_count']}")
print(f"   High Relevance: {result['hierarchy']['high']}")
print(f"   Medium Relevance: {result['hierarchy']['medium']}")
print(f"   Low Relevance: {result['hierarchy']['low']}")

print(f"\nAI Analysis:")
print(result['analysis'])

print(f"\nSource Documents ({len(result['source_documents'])})")
for i, doc in enumerate(result['source_documents'][:5], 1):
    print(f"{i}. {doc.metadata.get('name', 'Unknown')} - {doc.metadata.get('path', 'Unknown')}")

## Performance Comparison

Compare different RAG approaches:

In [ ]:
import time

def compare_rag_approaches(question: str):
    """Compare different RAG approaches for performance and quality."""
    
    approaches = {
        "Basic RAG": base_retriever,
        "Multi-Query RAG": multi_query_retriever,
        "Compressed RAG": compression_retriever,
        "Ensemble RAG": ensemble_retriever
    }
    
    results = {}
    
    for name, retriever in approaches.items():
        start_time = time.time()
        try:
            docs = retriever.invoke(question)
            duration = time.time() - start_time
            
            results[name] = {
                "documents": len(docs),
                "duration": duration,
                "avg_doc_length": sum(len(doc.page_content) for doc in docs) / len(docs) if docs else 0,
                "success": True
            }
        except Exception as e:
            results[name] = {
                "error": str(e),
                "success": False
            }
    
    return results

# Compare approaches
comparison_query = "project status update"
comparison = compare_rag_approaches(comparison_query)

print(f"Performance Comparison for: '{comparison_query}'")
print("\n" + "-"*60)

for approach, metrics in comparison.items():
    if metrics['success']:
        print(f"{approach:20} | {metrics['documents']:3d} docs | {metrics['duration']:5.2f}s | {metrics['avg_doc_length']:6.0f} chars")
    else:
        print(f"{approach:20} | ERROR: {metrics['error']}")

print("-"*60)

## Best Practices and Recommendations

### When to Use Each Pattern:

1. **Multi-Query RAG**: Complex questions requiring comprehensive coverage
2. **Contextual Compression**: Large documents with specific information needs
3. **Hierarchical RAG**: When document relevance varies significantly
4. **Self-Querying**: Complex user queries that need optimization
5. **Ensemble RAG**: Critical applications requiring maximum accuracy

### Performance Considerations:

- **Latency**: Basic RAG < Compressed < Multi-Query < Ensemble
- **Accuracy**: Basic RAG < Multi-Query < Compressed < Ensemble
- **Cost**: Basic RAG < Compressed < Multi-Query < Ensemble

### Production Deployment:

1. **Caching**: Implement aggressive caching for repeated queries
2. **Async Processing**: Use async operations for better throughput
3. **Load Balancing**: Distribute requests across multiple instances
4. **Monitoring**: Track performance metrics and error rates

## Next Steps

Explore more advanced patterns:
- **[Enterprise Workflows](03-enterprise-workflows.ipynb)**: Production deployment patterns
- **[Multi-Modal Analysis](04-multimodal-analysis.ipynb)**: Working with different document types
- **[Security & Compliance](05-security-compliance.ipynb)**: Enterprise security patterns